# Student Management System

## Import and create student database

- We will be using data from 'https://raw.githubusercontent.com/ShapeLab/ZooidsCompositePhysicalizations/master/Zooid_Vis/bin/data/student-dataset.csv'

### Import required libraries

In [1]:
import sqlite3
import pandas as pd

In [2]:
student_data = pd.read_csv('https://raw.githubusercontent.com/ShapeLab/ZooidsCompositePhysicalizations/master/Zooid_Vis/bin/data/student-dataset.csv')
student_data = student_data.drop('ethnic.group', axis=1)
# student_data.info()
col_with_dtype = []

for col in student_data.columns:
    col_with_dtype.append([col, student_data[col].dtype])

In [3]:
for i in range(len(col_with_dtype)):
    row = col_with_dtype[i]
    
    row[0] = ('_'.join(row[0].split('.'))).upper()
    
    if row[1] == int:
        row[1] = 'INT'
    elif row[1] == float:
        row[1] = 'REAL'
    else:
        row[1] = 'CHAR(30)'

In [4]:
# NBVAL_IGNORE_OUTPUT
col_with_dtype.pop(0)
col_with_dtype

[['NAME', 'CHAR(30)'],
 ['NATIONALITY', 'CHAR(30)'],
 ['CITY', 'CHAR(30)'],
 ['LATITUDE', 'REAL'],
 ['LONGITUDE', 'REAL'],
 ['GENDER', 'CHAR(30)'],
 ['AGE', 'INT'],
 ['ENGLISH_GRADE', 'REAL'],
 ['MATH_GRADE', 'REAL'],
 ['SCIENCES_GRADE', 'REAL'],
 ['LANGUAGE_GRADE', 'REAL'],
 ['PORTFOLIO_RATING', 'INT'],
 ['COVERLETTER_RATING', 'REAL'],
 ['REFLETTER_RATING', 'INT']]

In [5]:
# NBVAL_IGNORE_OUTPUT
template = "CREATE TABLE STUDENT (ID INTEGER PRIMARY KEY, {values});"
values = ""

for row in col_with_dtype:
    values += '{col_name} {dtype}, '.format(col_name=row[0], dtype=row[1])
values = values[:-2]
template = template.format(values=values)
template

'CREATE TABLE STUDENT (ID INTEGER PRIMARY KEY, NAME CHAR(30), NATIONALITY CHAR(30), CITY CHAR(30), LATITUDE REAL, LONGITUDE REAL, GENDER CHAR(30), AGE INT, ENGLISH_GRADE REAL, MATH_GRADE REAL, SCIENCES_GRADE REAL, LANGUAGE_GRADE REAL, PORTFOLIO_RATING INT, COVERLETTER_RATING REAL, REFLETTER_RATING INT);'

In [6]:
queries_to_exec = []
col_names = [col for col, _ in col_with_dtype]

for _, row in student_data.iterrows():
    queries_to_exec.append(f'INSERT INTO STUDENT {tuple(col_names)} VALUES {tuple(row.to_numpy()[1:])}')

In [7]:
# NBVAL_IGNORE_OUTPUT
try:

    # Connect to DB and create a cursor
    sqliteConnection = sqlite3.connect('student.db')
    cursor = sqliteConnection.cursor()
    print('> DB Connection Initalized!')

    # Write a query and execute it with cursor
    query = 'select sqlite_version();'
    cursor.execute(query)
    
    result = cursor.fetchall()
    
    print('> SQLite Version is {}'.format(result))
    
    
    print("> Deleting table 'STUDENT' if previously existed!")
    sqliteConnection.execute("DROP TABLE IF EXISTS STUDENT")
    
    print("> Creating Table 'STUDENT'!")
    # Creating table
    table = template
    sqliteConnection.execute(table)
    
    print(">> Adding rows!")
    for query in queries_to_exec:
        sqliteConnection.execute(query)
    
    print(">> Commiting rows!")
    sqliteConnection.commit()
    # Fetch and output result
    
    # Close the cursor
    cursor.close()
    print("> DB Modified Successfully!")

# Handle errors
except sqlite3.Error as error:
    print('Error occurred - ', error)

finally:
    if sqliteConnection:
        sqliteConnection.close()
        print('> SQLite Connection closed')

> DB Connection Initalized!
> SQLite Version is [('3.46.0',)]
> Deleting table 'STUDENT' if previously existed!
> Creating Table 'STUDENT'!
>> Adding rows!
>> Commiting rows!
> DB Modified Successfully!
> SQLite Connection closed


In [8]:
# NBVAL_IGNORE_OUTPUT
def get_results(query):
    results = []
    
    try:
        # Connect to DB and create a cursor
        sqliteConnection = sqlite3.connect('student.db')
        cursor = sqliteConnection.cursor()

        # Fetch and output result
        cursor.execute(query)

        col_names = [i[0] for i in cursor.description]
        
        results = cursor.fetchall()
        
        final_results = [{col: res for col, res in zip(col_names, result)} for result in results]

        results = final_results

        # Close the cursor
        cursor.close()

    # Handle errors
    except sqlite3.Error as error:
        results = f'Error occurred while retriving the requested data.'

    finally:
        if sqliteConnection:
            sqliteConnection.close()
            
    return results

# statement = '''SELECT * FROM STUDENT WHERE STUDENT.NAME = "Kiana Lor"'''
statement = '''SELECT COUNT(DISTINCT(NATIONALITY)) as UNIQUE_CONTRIES FROM STUDENT WHERE STUDENT.GENDER = "F"'''
# statement = '''SELECT * FROM STUDENT'''

print(f"RESULT: {get_results(statement)}")

RESULT: [{'UNIQUE_CONTRIES': 23}]


## Create a Pipeline for our management system

### Import the required libraries

In [9]:
# NBVAL_IGNORE_OUTPUT
import os
from typing import List, Dict, Optional, Any

from haystack import Document, component, Pipeline
from haystack.components.generators.chat import HuggingFaceAPIChatGenerator
from haystack.dataclasses import ChatMessage
from haystack.components.builders import ChatPromptBuilder

from haystack.utils import Secret
from haystack.utils.hf import HFGenerationAPIType

In [10]:
@component
class SQLDocumentRetriver:
    """
    A component that retirves the documents from SQL given a query.
    """
    @component.output_types(documents=list)
    def run(self, query:List[ChatMessage]):
        query = query[0].content
        query = query.split(';')[0]

        docs = [Document(content=str(row)) for row in get_results(query)]
        return {"documents": docs}

In [11]:
@component
class CustomGenerator:
    def __init__(self, genComponent):
        self.genComponent = genComponent

    @component.output_types(replies=List[ChatMessage])
    def run(self, prompt: List[ChatMessage], generation_kwargs: Optional[Dict[str, Any]] = None):
        
        result = self.genComponent.run(prompt)
        
        return result

In [12]:
template1 = [
    ChatMessage.from_system("""
    You are a skillful, helpful, knowledgable and honest chatbot that generates optimized SQLite3 queries.
    
    Given a question, you create a single optimized SQLite3 query that fetches the data from 'STUDENT' table. 
    Always start your answer with the query. Do not generate any note or anything other than query. 
    You will only fetch top 5 entries using (LIMIT) unless specified by user.
    
    Table 'STUDENT':
    {{table_template}}
    
    Question: Count the number of students in the database who are male.
    Answer: SELECT COUNT(NAME) as NUMBER_OF_MALE_STUDENTS FROM STUDENT WHERE STUDENT.GENDER = "M";
    
    Question: How many students are male ?
    Answer: SELECT COUNT(NAME) as NUMBER_OF_MALE_STUDENTS FROM STUDENT WHERE STUDENT.GENDER = "M";
    
    Question: Count the number of male students and display them as 'NO_OF_MALE_STUDS' who are from Canada.
    Answer: SELECT COUNT(NAME) as NO_OF_MALE_STUDS FROM STUDENT WHERE STUDENT.GENDER = "M" AND STUDENT.NATIONALITY = 'Canada';
    """),
    
    ChatMessage.from_user("""
    Question: {{question}}.
    Answer: 
    """)
]


template2 = [
    ChatMessage.from_system("""
    You are a helpful and honest chatbot that assists user in retriving the data from provided documents.
    
    DOCUMENTS:
    {'TOTAL_NUMBER_OF_STUDENTS': 100}
    
    Question: How many students are there in my class ?
    Answer: There are 100 students in your class.

    DOCUMENTS:
    {'SECOND_HIGHEST': "Andrew"}
    
    Question: Who has second highest score in Maths ?
    Answer: Andrew has second highest score in Maths.


    DOCUMENTS:
    [{"Name": "Dog"}, {"Name": "Apple"}, {"Name": "Ball"}, {"Name": "Cat"}]
    
    Question: List top 4 students to get highest marks ?
    Answer: The top 4 students to get highest marks are: 1. Dog\n2. Apple\n3. Ball\n4. Cat.

    """),

    ChatMessage.from_user("""
    Given the following information from documents, contruct a proper sentence for human understanding.
    Do not assume anything given the question. 
    The document contains all the information needed to answer the question.
    
    DOCUMENTS:
    {% for document in documents %}
        {{ document.content }}
        
    {% endfor %}
    
    Question: {{question}}
    Answer: 
    """)
]

query_prompt_builder = ChatPromptBuilder(template=template1)
answer_prompt_builder = ChatPromptBuilder(template=template2)

In [13]:
retriver = SQLDocumentRetriver()

In [15]:
api_type = HFGenerationAPIType.SERVERLESS_INFERENCE_API
generator =HuggingFaceAPIChatGenerator(api_type=api_type,
                                       api_params={"model": "mistralai/Mistral-7B-Instruct-v0.3"},
                                       token=Secret.from_env_var("HF_TOKEN"))

In [16]:
retriverGen = CustomGenerator(generator)
outputGen = CustomGenerator(generator)

In [17]:
# NBVAL_IGNORE_OUTPUT
student_mgmt_pipeline = Pipeline()

student_mgmt_pipeline.add_component("sql_retriver", retriver)
student_mgmt_pipeline.add_component("query_generator", retriverGen)
student_mgmt_pipeline.add_component("answer_generator", outputGen)
student_mgmt_pipeline.add_component("query_prompt_builder", query_prompt_builder)
student_mgmt_pipeline.add_component("answer_prompt_builder", answer_prompt_builder)

In [18]:
# NBVAL_IGNORE_OUTPUT
student_mgmt_pipeline.connect("query_prompt_builder.prompt", "query_generator")
student_mgmt_pipeline.connect("query_generator.replies", "sql_retriver")
student_mgmt_pipeline.connect("sql_retriver.documents", "answer_prompt_builder")
student_mgmt_pipeline.connect("answer_prompt_builder.prompt", "answer_generator")

🚅 Components
  - sql_retriver: SQLDocumentRetriver
  - query_generator: CustomGenerator
  - answer_generator: CustomGenerator
  - query_prompt_builder: ChatPromptBuilder
  - answer_prompt_builder: ChatPromptBuilder
🛤️ Connections
  - sql_retriver.documents -> answer_prompt_builder.documents (list)
  - query_generator.replies -> sql_retriver.query (List[ChatMessage])
  - query_prompt_builder.prompt -> query_generator.prompt (List[ChatMessage])
  - answer_prompt_builder.prompt -> answer_generator.prompt (List[ChatMessage])

### Running Student Management System

* Question 1: Name the student that has highest grades across all subjects

In [19]:
# NBVAL_IGNORE_OUTPUT
question = "Name the student that has highest grades across all subjects"

response = student_mgmt_pipeline.run({
    "query_prompt_builder": {"table_template": template ,"question": question},
    "answer_prompt_builder": {"question": question}
})

print(response['answer_generator']['replies'][0].content)

The student with the highest grades across all subjects is Kiana Lor.


* Question 2: List the names of 10 student who have highest grades across all subjects.

In [20]:
# NBVAL_IGNORE_OUTPUT
question = "List the names of 10 student who have highest grades across all subjects."

response = student_mgmt_pipeline.run({
    "query_prompt_builder": {"table_template": template, "question": question},
    "answer_prompt_builder": {"question": question}
})

print(response['answer_generator']['replies'][0].content)

The top 10 students with the highest grades across all subjects are Michael Griffin, Brittany Sath, Meghan Arnold, Faviola Soto, Rylee Onstott, Joshua Mesan, Andrew Bishop, Troy White, Vittoria Faulkner, and Patricia Rockhold. These students have all obtained a grade of 4.0 in English, and their grades in Math, Sciences, and Language vary, but they all have a consistent high grade in their Portfolio, Cover Letter, and Reference Letter ratings.


* Question 3: list the names of 10 student who have lowest grade in language 

In [21]:
# NBVAL_IGNORE_OUTPUT
question = "list the names of 10 student who have lowest grade in language "

response = student_mgmt_pipeline.run({
    "query_prompt_builder": {"table_template": template, "question": question},
    "answer_prompt_builder": {"question": question}
})

print(response['answer_generator']['replies'][0].content)

The students with the ten lowest grades in language, according to the given information, are:

1. Lee Dong
2. Rojesh Her
3. David Weber
4. Preston Suarez
5. Joey Abreu
6. Nicole Torres-Valadez
7. Devon Miranda
8. Issac Mata
9. Brooke Cazares
10. Kiana Lor


* Question 4: List all the subject grades of 'Nicole Torres-Valadez' and give a summary

In [22]:
# NBVAL_IGNORE_OUTPUT
question = "List all the subject grades of 'Nicole Torres-Valadez' and give a summary"

response = student_mgmt_pipeline.run({
    "query_prompt_builder": {"table_template": template, "question": question},
    "answer_prompt_builder": {"question": question}
})

print(response['answer_generator']['replies'][0].content)

Nicole Torres-Valadez's subject grades are as follows:
- English Grade: 3.9
- Math Grade: 3.1
- Sciences Grade: 3.0
- Language Grade: 1.0

Her highest subject grade is English with 3.9 and her lowest grade is Language with 1.0.


* Question5: What is the average grades of all the students in english subject

In [23]:
# NBVAL_IGNORE_OUTPUT
question = "What is the average grades of all the students in english subject"

response = student_mgmt_pipeline.run({
    "query_prompt_builder": {"table_template": template, "question": question},
    "answer_prompt_builder": {"question": question}
})

print(response['answer_generator']['replies'][0].content)

The average grade of all the students in the English subject is 3.3697068403908794.


* Question 6: Given my hometown is Brazil, how many students are from my hometown ?

In [24]:
# NBVAL_IGNORE_OUTPUT
question = "Given my hometown is Brazil, how many students are from my hometown ?"

response = student_mgmt_pipeline.run({
    "query_prompt_builder": {"table_template": template, "question": question},
    "answer_prompt_builder": {"question": question}
})

print(response['answer_generator']['replies'][0].content)

There are 9 students from your hometown, which is Brazil.


* Question 7: Which student has low performance in english language ?

In [25]:
# NBVAL_IGNORE_OUTPUT
question = "Which student has low performance in english language ?"

response = student_mgmt_pipeline.run({
    "query_prompt_builder": {"table_template": template, "question": question},
    "answer_prompt_builder": {"question": question}
})

print(response['answer_generator']['replies'][0].content)

Abdul Jabbaar el-Abdul has low performance in the English language, with a grade of 1.5.


### Application Endpoint

In [26]:
# NBVAL_SKIP
print("Entering Application Mode. [press ':q' to exit.]")

while (question:= input("> User: ")) != ':q':
    response = student_mgmt_pipeline.run({
        "query_prompt_builder": {"table_template": template, "question": question},
        "answer_prompt_builder": {"question": question}
    })

    print(f"> Bot: {response['answer_generator']['replies'][0].content}")

Entering Application Mode. [press ':q' to exit.]


> User:  How many students are in my class ?


> Bot: There are 307 students in the class.


> User:  :q
